In [1]:
print("Hello World!")

Hello World!


In [2]:
import urllib.request
import os

url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt"
file_path = "the-verdict.txt"

# check if the file exists
if not os.path.exists(file_path):
    # download the file
    urllib.request.urlretrieve(url, file_path)

print("File downloaded successfully!")

File downloaded successfully!


In [3]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("total number of characters in the text: ", len(raw_text))
      
print(raw_text[:99])

total number of characters in the text:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(len(preprocessed))

print(preprocessed[:30])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [7]:
# now we have to find a mapping between word and ID of words

all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(f"vocab size: {vocab_size}")

vocab = {token: integer for integer, token in enumerate(all_words)}

# print first 50 for debug purposes
for i, item in enumerate(vocab.items()):
    if i > 50:
        break
    print(item)


vocab size: 1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [11]:
# define a simple tokenizer

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        return [self.str_to_int[w] for w in preprocessed if w in self.str_to_int]
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'([,.:;?_!"()\']|--|\s)', r'\1', text)
        return text
    
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)


[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [12]:
print(tokenizer.decode(ids))

" It ' s the last he painted , you know , " Mrs . Gisburn said with pardonable pride .


In [14]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token: integer for integer, token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


In [19]:
# print the last 5 elements
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [22]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int else "<|unk|>"
            for item in preprocessed
        ]
        return [self.str_to_int[item] for item in preprocessed]
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[id] for id in ids])
        text = re.sub(r'([,.:;?_!"()\']|--|\s)', r'\1', text)
        return text
    
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terracees of the palace."
text = " <|endoftext|> ".join([text1, text2])

print(text)



Hello, do you like tea? <|endoftext|> In the sunlit terracees of the palace.


In [23]:
tokenizer = SimpleTokenizerV2(vocab)

print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 1131, 722, 988, 1131, 7]


In [24]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|> , do you like tea ? <|endoftext|> In the sunlit <|unk|> of the <|unk|> .


In [25]:
from importlib.metadata import version
import tiktoken

print("tiktoken version: ", version("tiktoken"))

tiktoken version:  0.8.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)



[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [28]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [30]:
# encode some unknown text and see the indexes
text_unknown = "Akwirw ier"

integers_unknown = tokenizer.encode(text_unknown)

print(integers_unknown)

strings_unknown = tokenizer.decode(integers_unknown)

print(strings_unknown)



[33901, 86, 343, 86, 220, 959]
Akwirw ier


In [41]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)

print(len(enc_text))

enc_sample = enc_text[50:]

5145


In [42]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:     {y}")


x: [290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


In [43]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(f"context: {context} -> desired: {desired}")

context: [290] -> desired: 4920
context: [290, 4920] -> desired: 2241
context: [290, 4920, 2241] -> desired: 287
context: [290, 4920, 2241, 287] -> desired: 257


In [44]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(f"context: {tokenizer.decode(context)} -> desired: {tokenizer.decode([desired])}")

context:  and -> desired:  established
context:  and established -> desired:  himself
context:  and established himself -> desired:  in
context:  and established himself in -> desired:  a
